## Game Prices Scraper Concept
Notebook to test of webscraper to aquire data of «watched» games. 

### Aquired Data
1. Steam web api
2. Will add other sites to make price comparison

### Import dependencies

In [2]:
from bs4 import BeautifulStoneSoup as sp
import requests
import pandas as pd
import json

### Get all games

In [5]:
# Make a request to retrieve the list of games
response = requests.get('https://api.steampowered.com/ISteamApps/GetAppList/v2/')

games = []

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()

    # Retrieve the list of apps from the response
    app_list = data['applist']['apps']

    # Create a dictionary to store game names and app IDs
    game_dict = {}

    # Iterate over each app and store the game name and app ID in the dictionary
    for app in app_list:
        game_name = app['name']
        app_id = app['appid']
        
        if game_name == '' or app_id == '':
            continue

        games.append({
            "name":game_name,
            "id":app_id
        })
else:
    print("Failed to retrieve the list of games.")


df_games = pd.DataFrame(games)
df_games.to_csv('steam_list.csv', sep=';', index=False)

### Get game details using Steam Api

In [8]:
import requests

game_name = "Dead Cells"  # Replace with the desired game name

response = requests.get(f'https://api.steampowered.com/ISteamApps/GetAppList/v2/')
if response.status_code == 200:
    game_list = response.json()['applist']['apps']
    game = next((app for app in game_list if app['name'].lower() == game_name.lower()), None)

    if game:
        app_id = game['appid']
        app_details_url = f'https://store.steampowered.com/api/appdetails?appids={app_id}'
        app_details_response = requests.get(app_details_url)
        if app_details_response.status_code == 200:
            app_details = app_details_response.json()[str(app_id)]
            success = app_details['success']
            if success:
                data = app_details['data']
                release_date = data['release_date']['date']
                print(f"Release Date: {release_date}")
                price_overview = data['price_overview']
                print(f"App ID: {app_id}")
                print(f"Price: {price_overview['final_formatted']}")

                if 'discount_percent' in price_overview:
                    discount_percent = price_overview['discount_percent']
                    discount_price = price_overview['final']
                    original_price = price_overview['initial']
                    print(f"On Sale: Yes")
                    print(f"Discount: {discount_percent}%")
                    print(f"Sale Price: {discount_price}")
                    print(f"Original Price: {original_price}")
                else:
                    print("On Sale: No")

                # Retrieve the game description
                game_description = data['detailed_description']
                print(f"Game Description: {game_description}")

                # Retrieve the image URL
                header_image = data['header_image']
                print(f"Image URL: {header_image}")
            else:
                print(f"Failed to retrieve data for app ID: {app_id}")
        else:
            print("Error occurred while retrieving app details:", app_details_response.status_code)
    else:
        print(f"Game '{game_name}' not found.")
else:
    print("Error occurred while retrieving game list:", response.status_code)


Release Date: 6 Aug, 2018
App ID: 588650
Price: 12,49€
On Sale: Yes
Discount: 50%
Sale Price: 1249
Original Price: 2499
Game Description: <h1>New DLC Available</h1><p><a href="https://store.steampowered.com/app/2101430/Dead_Cells_Return_to_Castlevania/" target="_blank" rel=""  id="dynamiclink_0" >https://store.steampowered.com/app/2101430/Dead_Cells_Return_to_Castlevania/</a><h2 class="bb_tag">Return to Castlevania DLC is available now!</h2></p><br><h1>About the Game</h1><img src="https://cdn.akamai.steamstatic.com/steam/apps/588650/extras/store-3-wall-jump.gif?t=1678188017" /><h2 class="bb_tag">Roguelite? Metroidvania? Roguevania!</h2>You grew up with the roguelikes, witnessed the rise of the roguelites and even the birth of the roguelite-lites?  We'd now like to present for your consideration our <strong>RogueVania</strong>, the illegitimate child of a modern <strong>Roguelite</strong> (<i>Rogue Legacy, Binding of Isaac, Enter the Gungeon, Spelunky,</i> etc.) and an old-school <stron